In [25]:
using POMDPs
using Random # for AbstractRNG
using POMDPModelTools
using Pkg
Pkg.add("JSON")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 %61.9 %  Updating registry at `~/.julia/registries/JuliaPOMDP`
  Updating git-repo `https://github.com/JuliaPOMDP/Registry`
 Resolving package versions...
 Installed Distances ─ v0.8.0
  Updating `~/.julia/environments/v1.0/Project.toml`
  [682c06a0] + JSON v0.20.0
  Updating `~/.julia/environments/v1.0/Manifest.toml`
  [b4f34e82] ↑ Distances v0.7.4 ⇒ v0.8.0


In [2]:
struct ChainMDP <: MDP{Int, Symbol}
    len::Int
    p_success::Float64
    discount::Float64
    theta::Int
end


In [3]:
function POMDPs.generate_s(p::ChainMDP, s::Int, a::Symbol, rng::AbstractRNG)
    if a == :right
        success = min(s+1, p.len)
        failure = max(s-1, 1)
    else # a == :left
        success = max(s-1, 1)
        failure = min(s+1, p.len)
    end
    if s + 1 == p.len
        return p.len 
    elseif  s == 2
        return 1
    end
    return rand(rng) < p.p_success ? success : failure
end


In [4]:
# theta = 10
function POMDPs.reward(p::ChainMDP, s::Int, a::Symbol)
    if s == 2
        return p.theta
    end
    if s + 1 == p.len
        return -p.theta
    end
    if s == 0 || s == p.len
        return 0
    end
    return -1
end

In [5]:
POMDPs.initialstate_distribution(m::ChainMDP) = Deterministic(4)

In [6]:
function POMDPs.isterminal(p::ChainMDP, s::Int)
    if s == 1
        return true
    end
    if s == p.len
        return true
    end
    return false
end        

In [7]:
using POMDPSimulators
using POMDPPolicies

ChainMDP() = ChainMDP(6+2,1.0,.9, 10)
m = ChainMDP()

# policy that maps every input to a right action
policy = FunctionPolicy(s->:right)

for (s, a, r) in stepthrough(m, policy, "s,a,r", max_steps=10)
    @show s
    @show a
    @show r
    render(m, (s,a,r))
    println()
end


s = 4
a = :right
r = -1

s = 5
a = :right
r = -1

s = 6
a = :right
r = -1

s = 7
a = :right
r = -10



In [8]:
using POMDPSimulators
using POMDPPolicies

ChainMDP() = ChainMDP(6+2,1.0,.9, 10)
m = ChainMDP()

# policy that maps every input to a left action
policy = FunctionPolicy(s->:left)

function POMDPs.initialstate_distribution(m::ChainMDP)
    return Deterministic(4)
end
for (s, a, r) in stepthrough(m, policy, "s,a,r", max_steps=10)
    # @show s
    # @show a
    # @show r
    render(m, (s,a,r))
    println("s,a,r:($s,$a,$r)")
end


s,a,r:(4,left,-1)
s,a,r:(3,left,-1)
s,a,r:(2,left,10)


In [9]:

ChainMDP() = ChainMDP(6+2,1.0,.9, 10)
m = ChainMDP()

# policy that maps every input to a right action
policy = FunctionPolicy(s->:right)

function POMDPs.initialstate_distribution(m::ChainMDP)
    return Deterministic(4)
end
for (s, a, r) in stepthrough(m, policy, "s,a,r", max_steps=10)
    # @show s
    # @show a
    # @show r
    render(m, (s,a,r))
    println("s,a,r:($s,$a,$r)")
end


s,a,r:(4,right,-1)
s,a,r:(5,right,-1)
s,a,r:(6,right,-1)
s,a,r:(7,right,-10)


In [10]:
n_agents = 1
agents = Any[]
mdps = Any[]
n_states = 6
n_actions = 2
epochs = 10
H = 10
actions = [1, 2]
action_map = Dict(1 => :left, :2 => :right)
rev_action_map = Dict(:left => 1, :right => 2)
states = 1:(n_states+2)
# setup agents
Q_tables = []
N_tables = zeros((n_agents, n_states+2, n_actions))
print(N_tables)
policies = []
theta = 10
# policy that maps every input to a right action
for i in 1:n_agents
    push!(Q_tables, Dict{Int32,Float32}())
    push!(policies, s->:right)
    push!(mdps, ChainMDP(n_states+2,.9,.9, theta))
end
# policy = s->:right

function update_Q(Q_table, s, a, r, sp, t)
    alpha=.95
    gamma=.95
    # print("s:$s, a:$a, sp:$sp, r:$r")
    Q_table[s][rev_action_map[a]] +=  alpha * (r + gamma * 
        findmax(Q_table[sp])[1] - Q_table[s][rev_action_map[a]])
    #println(Q_table)
end
function POMDPs.initialstate_distribution(m::ChainMDP)
    return Deterministic(Int64((n_states+2)/2))
end
function run_chain!(;policies, mdps, reward_reveal_condition, true_mdp, update_Q, n_agents, n_states,
                    Q_tables, N_tables, epochs, steps)
    for e in 1:epochs
        agents = []
        for i in 1:n_agents
            m = mdps[i]
            push!(agents, 
                  Iterators.Stateful(stepthrough(m, FunctionPolicy(policies[i]), "s,a,r,sp,t", max_steps=steps)))
        end

        println("epoch: $e")
        done = false
        t = 0
        while ! done
           done = true
            
           for i in 1:n_agents
                if isempty(agents[i])
                    println("agent $i is done")
                    continue
                end
                res = popfirst!(agents[i])
                r = res[:r]
                if reward_reveal_condition(r)
                    for mdp in mpds
                        mdp = true_mdp
                    end
                end
                #println("before update: N_table: $N_tables")
                N_tables[i,res[:s],rev_action_map[res[:a]]] += 1
                #println("after update:  N_table: $N_tables")
                # println("update Q")
                update_Q(Q_tables[i],res...)
                t = res[:t]
                println("t: $t, print agent $i result: $res")
          
           end
           # println("t:$t")
           for i in 1:n_agents
                if ! isempty(agents[i])
                    done = false
                end
            end
        end
    end
end
run_chain!(policies=policies,
           mdps=mdps,  
           true_mdp=mdps[1],
           reward_reveal_condition= r -> false,
           update_Q=update_Q,
           n_agents=n_agents,
           n_states=n_states,
           Q_tables=Q_tables,
           N_tables=N_tables,
           epochs=epochs,
           steps=H)

[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]

[0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]epoch: 1


KeyError: KeyError: key 4 not found

In [11]:
function square(x)
    return x * x
end

vals = Dict(1 => 4, 2 => 5, 3 => 6)
Dict(key=> square(val) for (key, val) in vals)

Dict{Int64,Int64} with 3 entries:
  2 => 25
  3 => 36
  1 => 16

In [12]:
A = [1 2; 3 4; 5 6]
B = A .+ 6
C = B .+ 6
D = hcat([A , B , C])

3×1 Array{Array{Int64,2},2}:
 [1 2; 3 4; 5 6]      
 [7 8; 9 10; 11 12]   
 [13 14; 15 16; 17 18]

In [13]:
D[2,:]

1-element Array{Array{Int64,2},1}:
 [7 8; 9 10; 11 12]

In [6]:
function ucb_pol(Q_tables, N_tables, i, actions, s)
    # return 
    if haskey(Q_tables[i], s)
       # println("ucb_pol:s:$s, N_tables:$N_tables")
       ucbs = Dict(key => sqrt(log(sum(N_tables[i][s,:])/N_tables[i][s,key])) 
                for (key,val) in Q_tables[i][s])
       # println("ucb: $(ucbs)")
       val, idx = findmax(Dict(key => Q_tables[i][s][key] + sqrt(log(sum(N_tables[i][s,:])/N_tables[i][s,key])) 
                for (key,val) in Q_tables[i][s])) # need to test
       return action_map[idx]
    else
       act = action_map[rand(actions, 1)[1]]
       # println("selected action $act") 
       return act
    end
end




ucb_pol (generic function with 1 method)

In [7]:
curry(f, x) = (xs...) -> f(x, xs...)

curry (generic function with 1 method)

In [16]:
# UCB

n_states = 10
curry(f, x) = (xs...) -> f(x, xs...)
Q_tables = []
mdps = Any[]
N_tables = zeros((n_agents, n_states+2, n_actions))
policies = []
epochs = 500
theta = 10
H = 10
# Setup Q table according to MDP

n_agents = 1
agents = Any[]
n_states = 10

function POMDPs.reward(p::ChainMDP, s::Int, a::Symbol)
    if s == 2
        return 10
    end
    if s + 1 == p.len
        return -10
    end
    if s == 0 || s == p.len
        return 0
    end
    return -1
end

#setup
for i in 1:n_agents
    push!(Q_tables, Dict{Int32, Dict{Int32, Float32}}())
    for state in states
      Q_tables[i][state] = Dict{Int32, Float32}()
      # print(Q_tables[i])
      for action in actions
         Q_tables[i][state][action] = 0 
      end
    end
    N_table = N_tables[i,:,:]
    push!(policies, curry(curry(curry(curry(ucb_pol, Q_tables), N_tables),i), actions))
    push!(mdps, ChainMDP(n_states+2,.9,.9, 10))
end

run_chain!(policies, mdps, theta, update_Q, n_agents,  n_states, Q_tables, N_tables, epochs, H)

MethodError: MethodError: no method matching run_chain!(::Array{Any,1}, ::Array{Any,1}, ::Int64, ::typeof(update_Q), ::Int64, ::Int64, ::Array{Any,1}, ::Array{Float64,3}, ::Int64, ::Int64)

In [17]:

n_agents = 10
Q_tables = [deepcopy(Q_tables[1]) for i in range(1,n_agents)]
policies = [deepcopy(policies[1]) for i in range(1,n_agents)]
mdps = [deepcopy(mdps[1]) for i in range(1,n_agents)]
N_tables = zeros((n_agents, n_states+2, n_actions))
run_chain!(policies, mdps, update_Q, n_agents,  n_states, Q_tables, N_tables, epochs, H)

MethodError: MethodError: no method matching run_chain!(::Array{getfield(Main, Symbol("##22#23")){getfield(Main, Symbol("##22#23")){getfield(Main, Symbol("##22#23")){getfield(Main, Symbol("##22#23")){typeof(ucb_pol),Array{Any,1}},Array{Float64,3}},Int64},Array{Int64,1}},1}, ::Array{ChainMDP,1}, ::typeof(update_Q), ::Int64, ::Int64, ::Array{Dict{Int32,Dict{Int32,Float32}},1}, ::Array{Float64,3}, ::Int64, ::Int64)

In [18]:
# paramter for seed sampling
using Distributions
theta = 10 * sign(rand(Bernoulli(0.5))-.5)
ntheta = - theta
print("theta: $theta, ntheta: $ntheta")
# Thompson sampling
#Should just be seed sampling, but you do it every step

theta: -10.0, ntheta: 10.0

In [4]:

using POMDPs
include("./ChainMDP.jl")
num_states = 10

mdp = PFChainMDP.PChainMDP(num_states+2,.9,.9, 10)
POMDPs.reward(mdp, 1, :left)

-1

In [8]:
function update_Q(Q_table, s, a, r, sp, t, rev_action_map)
    alpha=.95
    gamma=.95
    # print("s:$s, a:$a, sp:$sp, r:$r")
    Q_table[s][rev_action_map[a]] +=  alpha * (r + gamma * 
        findmax(Q_table[sp])[1] - Q_table[s][rev_action_map[a]])
    #println(Q_table)
end

update_Q (generic function with 1 method)

In [9]:
# UCB
using Distributions
include("./Agent.jl")
num_agents = 5
agents = Any[]
num_states = 20
num_actions = 2
epochs = 30
H = floor(Int, 3 * num_states / 2)
actions = [1, 2]
action_map = Dict(1 => :left, :2 => :right)
rev_action_map = Dict(:left => 1, :right => 2)
states = 1:(num_states+2)

theta = 10
# Setup Q table according to MDP



#setup
agmdp = PFChainMDP.PChainMDP(num_states+2,1.0,.9, theta * sign(rand(Bernoulli(0.5))-.5))
(Q_tables, N_tables, policies, mdps) = PFAgent.setup_agents(states, num_states, num_agents,
                                                            actions, num_actions, ucb_pol, agmdp)
true_mdp = PFChainMDP.PChainMDP(num_states+2,1.0,.9, theta)

function chain_found_target(r)
   if r == -10 || r == 10
        return true
   end
   return false
end

PFAgent.run_chain!(
           policies=policies,
           found_target=chain_found_target,
           true_mdp=true_mdp,
           update_Q=update_Q,
           n_agents=num_agents,
           n_states=num_states,
           Q_tables=Q_tables,
           N_tables=N_tables,
           epochs=epochs,
           steps=H,
           rev_action_map=rev_action_map,
           stop_early=false)

t: 1, print agent 1 result: (s = 12, a = :right, r = -1, sp = 13, t = 1)
t: 2, print agent 1 result: (s = 13, a = :right, r = -1, sp = 14, t = 2)
t: 1, print agent 2 result: (s = 12, a = :right, r = -1, sp = 13, t = 1)
t: 3, print agent 1 result: (s = 14, a = :right, r = -1, sp = 15, t = 3)
t: 2, print agent 2 result: (s = 13, a = :right, r = -1, sp = 14, t = 2)
t: 1, print agent 3 result: (s = 12, a = :right, r = -1, sp = 13, t = 1)
t: 4, print agent 1 result: (s = 15, a = :right, r = -1, sp = 16, t = 4)
t: 3, print agent 2 result: (s = 14, a = :right, r = -1, sp = 15, t = 3)
t: 2, print agent 3 result: (s = 13, a = :right, r = -1, sp = 14, t = 2)
t: 1, print agent 4 result: (s = 12, a = :right, r = -1, sp = 13, t = 1)
t: 5, print agent 1 result: (s = 16, a = :right, r = -1, sp = 17, t = 5)
t: 4, print agent 2 result: (s = 15, a = :right, r = -1, sp = 16, t = 4)
t: 3, print agent 3 result: (s = 14, a = :right, r = -1, sp = 15, t = 3)
t: 2, print agent 4 result: (s = 13, a = :right, r 

In [42]:

include("./Agent.jl")
# Thompson sampling

# Solution

# solve mdp in both directions
# Then each agent will pick on of the policies.
# common

num_agents = 1
epochs = 500
num_states = 30
states = 1:(num_states+2)
H = floor(Int, 3 * num_states / 2)

theta = 10

true_mdp = PFChainMDP.PChainMDP(num_states+2,1.0,.9, theta)

(Q_tables1, N_tables1, policies1, mdps1) = PFAgent.setup_agents(states, num_states, num_agents,
                                                            actions, num_actions, ucb_pol, true_mdp)
PFAgent.run_chain!(policies=policies,
           found_target=chain_found_target,
           true_mdp=true_mdp,
           update_Q=update_Q,
           n_agents=num_agents,
           n_states=num_states,
           Q_tables=Q_tables1,
           N_tables=N_tables1,
           epochs=epochs,
           steps=H,
           rev_action_map=rev_action_map,
           stop_early=false)

theta = -10

true_mdp = PFChainMDP.PChainMDP(num_states+2,1.0,.9, theta)
(Q_tables2, N_tables2, policies2, mdps2) = PFAgent.setup_agents(states, num_states, num_agents,
                                                            actions, num_actions, ucb_pol, true_mdp)
PFAgent.run_chain!(policies=policies,
           found_target=chain_found_target,
           true_mdp=true_mdp,
           update_Q=update_Q,
           n_agents=num_agents,
           n_states=num_states,
           Q_tables=Q_tables2,
           N_tables=N_tables2,
           epochs=epochs,
           steps=H,
           rev_action_map=rev_action_map,
           stop_early=false)

# Now we need a policy which randomly picks between the 2
function thomp_pol_func(Q_tables, N_tables, i, actions, s)
    # passed in i ignored, kept for consistency
    i = rand(1:length(Q_tables))
    # return 
    if haskey(Q_tables[i], s)
       # println("ucb: $(ucbs)")
       val, idx = findmax(Q_tables[i][s])
       # print("$(Q_tables[i][s])")
       return action_map[idx]
    else
       act = action_map[rand(actions, 1)[1]]
       # println("selected action $act") 
       return act
    end
end
#Now we run with that policy
using JSON
println("Q1:$(json(Q_tables1[1],2))")
println("Q2: $(json(Q_tables2[1],2))")
true_mdp = PFChainMDP.PChainMDP(num_states+2,1.0,.9, theta)


num_agents = 2
thomp_policies = []
Q_tables_thomp = Dict(1 => Q_tables1[1], 2 => Q_tables2[1])
N_tables_thomp = Dict(1 => N_tables1[1], 2 => N_tables2[1])
push!(thomp_policies, curry(curry(curry(curry(thomp_pol_func, Q_tables_thomp), N_tables_thomp),0), actions))
push!(thomp_policies, curry(curry(curry(curry(thomp_pol_func, Q_tables_thomp), N_tables_thomp),0), actions))

(Q_tables, N_tables, policies, mdps) = PFAgent.setup_agents(states, num_states, num_agents,
                                                            actions, num_actions, ucb_pol, true_mdp)
PFAgent.run_chain!(
           policies=thomp_policies,
           found_target=chain_found_target,
           true_mdp=true_mdp,
           update_Q=update_Q,
           n_agents=num_agents,
           n_states=num_states,
           Q_tables=Q_tables,
           N_tables=N_tables,
           epochs=epochs,
           steps=H,
           rev_action_map=rev_action_map,
           stop_early=true)

e: 2, t: 1, print agent 1 result: (s = 17, a = :right, r = -1, sp = 18, t = 1)
e: 2, t: 2, print agent 1 result: (s = 18, a = :right, r = -1, sp = 19, t = 2)
e: 2, t: 3, print agent 1 result: (s = 19, a = :right, r = -1, sp = 20, t = 3)
e: 2, t: 4, print agent 1 result: (s = 20, a = :right, r = -1, sp = 21, t = 4)
e: 2, t: 5, print agent 1 result: (s = 21, a = :right, r = -1, sp = 22, t = 5)
e: 2, t: 6, print agent 1 result: (s = 22, a = :right, r = -1, sp = 23, t = 6)
e: 2, t: 7, print agent 1 result: (s = 23, a = :right, r = -1, sp = 24, t = 7)
e: 2, t: 8, print agent 1 result: (s = 24, a = :right, r = -1, sp = 25, t = 8)
e: 2, t: 9, print agent 1 result: (s = 25, a = :right, r = -1, sp = 26, t = 9)
e: 2, t: 10, print agent 1 result: (s = 26, a = :right, r = -1, sp = 27, t = 10)
e: 2, t: 11, print agent 1 result: (s = 27, a = :right, r = -1, sp = 28, t = 11)
e: 2, t: 12, print agent 1 result: (s = 28, a = :right, r = -1, sp = 29, t = 12)
e: 2, t: 13, print agent 1 result: (s = 29, a 

e: 18, t: 9, print agent 1 result: (s = 25, a = :right, r = -1, sp = 26, t = 9)
e: 18, t: 10, print agent 1 result: (s = 26, a = :right, r = -1, sp = 27, t = 10)
e: 18, t: 11, print agent 1 result: (s = 27, a = :right, r = -1, sp = 28, t = 11)
e: 18, t: 12, print agent 1 result: (s = 28, a = :right, r = -1, sp = 29, t = 12)
e: 18, t: 13, print agent 1 result: (s = 29, a = :right, r = -1, sp = 30, t = 13)
e: 18, t: 14, print agent 1 result: (s = 30, a = :right, r = -1, sp = 31, t = 14)
e: 18, t: 15, print agent 1 result: (s = 31, a = :right, r = -10, sp = 32, t = 15)
e: 20, t: 1, print agent 1 result: (s = 17, a = :right, r = -1, sp = 18, t = 1)
e: 20, t: 2, print agent 1 result: (s = 18, a = :right, r = -1, sp = 19, t = 2)
e: 20, t: 3, print agent 1 result: (s = 19, a = :right, r = -1, sp = 20, t = 3)
e: 20, t: 4, print agent 1 result: (s = 20, a = :right, r = -1, sp = 21, t = 4)
e: 20, t: 5, print agent 1 result: (s = 21, a = :right, r = -1, sp = 22, t = 5)
e: 20, t: 6, print agent 1 

e: 36, t: 1, print agent 1 result: (s = 17, a = :right, r = -1, sp = 18, t = 1)
e: 36, t: 2, print agent 1 result: (s = 18, a = :right, r = -1, sp = 19, t = 2)
e: 36, t: 3, print agent 1 result: (s = 19, a = :right, r = -1, sp = 20, t = 3)
e: 36, t: 4, print agent 1 result: (s = 20, a = :right, r = -1, sp = 21, t = 4)
e: 36, t: 5, print agent 1 result: (s = 21, a = :right, r = -1, sp = 22, t = 5)
e: 36, t: 6, print agent 1 result: (s = 22, a = :right, r = -1, sp = 23, t = 6)
e: 36, t: 7, print agent 1 result: (s = 23, a = :right, r = -1, sp = 24, t = 7)
e: 36, t: 8, print agent 1 result: (s = 24, a = :right, r = -1, sp = 25, t = 8)
e: 36, t: 9, print agent 1 result: (s = 25, a = :right, r = -1, sp = 26, t = 9)
e: 36, t: 10, print agent 1 result: (s = 26, a = :right, r = -1, sp = 27, t = 10)
e: 36, t: 11, print agent 1 result: (s = 27, a = :right, r = -1, sp = 28, t = 11)
e: 36, t: 12, print agent 1 result: (s = 28, a = :right, r = -1, sp = 29, t = 12)
e: 36, t: 13, print agent 1 result

e: 52, t: 8, print agent 1 result: (s = 24, a = :right, r = -1, sp = 25, t = 8)
e: 52, t: 9, print agent 1 result: (s = 25, a = :right, r = -1, sp = 26, t = 9)
e: 52, t: 10, print agent 1 result: (s = 26, a = :right, r = -1, sp = 27, t = 10)
e: 52, t: 11, print agent 1 result: (s = 27, a = :right, r = -1, sp = 28, t = 11)
e: 52, t: 12, print agent 1 result: (s = 28, a = :right, r = -1, sp = 29, t = 12)
e: 52, t: 13, print agent 1 result: (s = 29, a = :right, r = -1, sp = 30, t = 13)
e: 52, t: 14, print agent 1 result: (s = 30, a = :right, r = -1, sp = 31, t = 14)
e: 52, t: 15, print agent 1 result: (s = 31, a = :right, r = -10, sp = 32, t = 15)
e: 54, t: 1, print agent 1 result: (s = 17, a = :right, r = -1, sp = 18, t = 1)
e: 54, t: 2, print agent 1 result: (s = 18, a = :right, r = -1, sp = 19, t = 2)
e: 54, t: 3, print agent 1 result: (s = 19, a = :right, r = -1, sp = 20, t = 3)
e: 54, t: 4, print agent 1 result: (s = 20, a = :right, r = -1, sp = 21, t = 4)
e: 54, t: 5, print agent 1 

e: 68, t: 15, print agent 1 result: (s = 31, a = :right, r = -10, sp = 32, t = 15)
e: 70, t: 1, print agent 1 result: (s = 17, a = :right, r = -1, sp = 18, t = 1)
e: 70, t: 2, print agent 1 result: (s = 18, a = :right, r = -1, sp = 19, t = 2)
e: 70, t: 3, print agent 1 result: (s = 19, a = :right, r = -1, sp = 20, t = 3)
e: 70, t: 4, print agent 1 result: (s = 20, a = :right, r = -1, sp = 21, t = 4)
e: 70, t: 5, print agent 1 result: (s = 21, a = :right, r = -1, sp = 22, t = 5)
e: 70, t: 6, print agent 1 result: (s = 22, a = :right, r = -1, sp = 23, t = 6)
e: 70, t: 7, print agent 1 result: (s = 23, a = :right, r = -1, sp = 24, t = 7)
e: 70, t: 8, print agent 1 result: (s = 24, a = :right, r = -1, sp = 25, t = 8)
e: 70, t: 9, print agent 1 result: (s = 25, a = :right, r = -1, sp = 26, t = 9)
e: 70, t: 10, print agent 1 result: (s = 26, a = :right, r = -1, sp = 27, t = 10)
e: 70, t: 11, print agent 1 result: (s = 27, a = :right, r = -1, sp = 28, t = 11)
e: 70, t: 12, print agent 1 resul

...

e: 284, t: 12, print agent 1 result: (s = 28, a = :right, r = -1, sp = 29, t = 12)
e: 284, t: 13, print agent 1 result: (s = 29, a = :right, r = -1, sp = 30, t = 13)
e: 284, t: 14, print agent 1 result: (s = 30, a = :right, r = -1, sp = 31, t = 14)
e: 284, t: 15, print agent 1 result: (s = 31, a = :right, r = 10, sp = 32, t = 15)
e: 286, t: 1, print agent 1 result: (s = 17, a = :right, r = -1, sp = 18, t = 1)
e: 286, t: 2, print agent 1 result: (s = 18, a = :right, r = -1, sp = 19, t = 2)
e: 286, t: 3, print agent 1 result: (s = 19, a = :right, r = -1, sp = 20, t = 3)
e: 286, t: 4, print agent 1 result: (s = 20, a = :right, r = -1, sp = 21, t = 4)
e: 286, t: 5, print agent 1 result: (s = 21, a = :right, r = -1, sp = 22, t = 5)
e: 286, t: 6, print agent 1 result: (s = 22, a = :right, r = -1, sp = 23, t = 6)
e: 286, t: 7, print agent 1 result: (s = 23, a = :right, r = -1, sp = 24, t = 7)
e: 286, t: 8, print agent 1 result: (s = 24, a = :right, r = -1, sp = 25, t = 8)
e: 286, t: 9, print 

e: 302, t: 3, print agent 1 result: (s = 19, a = :right, r = -1, sp = 20, t = 3)
e: 302, t: 4, print agent 1 result: (s = 20, a = :right, r = -1, sp = 21, t = 4)
e: 302, t: 5, print agent 1 result: (s = 21, a = :right, r = -1, sp = 22, t = 5)
e: 302, t: 6, print agent 1 result: (s = 22, a = :right, r = -1, sp = 23, t = 6)
e: 302, t: 7, print agent 1 result: (s = 23, a = :right, r = -1, sp = 24, t = 7)
e: 302, t: 8, print agent 1 result: (s = 24, a = :right, r = -1, sp = 25, t = 8)
e: 302, t: 9, print agent 1 result: (s = 25, a = :right, r = -1, sp = 26, t = 9)
e: 302, t: 10, print agent 1 result: (s = 26, a = :right, r = -1, sp = 27, t = 10)
e: 302, t: 11, print agent 1 result: (s = 27, a = :right, r = -1, sp = 28, t = 11)
e: 302, t: 12, print agent 1 result: (s = 28, a = :right, r = -1, sp = 29, t = 12)
e: 302, t: 13, print agent 1 result: (s = 29, a = :right, r = -1, sp = 30, t = 13)
e: 302, t: 14, print agent 1 result: (s = 30, a = :right, r = -1, sp = 31, t = 14)
e: 302, t: 15, pri

e: 326, t: 5, print agent 1 result: (s = 21, a = :right, r = -1, sp = 22, t = 5)
e: 326, t: 6, print agent 1 result: (s = 22, a = :right, r = -1, sp = 23, t = 6)
e: 326, t: 7, print agent 1 result: (s = 23, a = :right, r = -1, sp = 24, t = 7)
e: 326, t: 8, print agent 1 result: (s = 24, a = :right, r = -1, sp = 25, t = 8)
e: 326, t: 9, print agent 1 result: (s = 25, a = :right, r = -1, sp = 26, t = 9)
e: 326, t: 10, print agent 1 result: (s = 26, a = :right, r = -1, sp = 27, t = 10)
e: 326, t: 11, print agent 1 result: (s = 27, a = :right, r = -1, sp = 28, t = 11)
e: 326, t: 12, print agent 1 result: (s = 28, a = :right, r = -1, sp = 29, t = 12)
e: 326, t: 13, print agent 1 result: (s = 29, a = :right, r = -1, sp = 30, t = 13)
e: 326, t: 14, print agent 1 result: (s = 30, a = :right, r = -1, sp = 31, t = 14)
e: 326, t: 15, print agent 1 result: (s = 31, a = :right, r = 10, sp = 32, t = 15)
e: 328, t: 1, print agent 1 result: (s = 17, a = :right, r = -1, sp = 18, t = 1)
e: 328, t: 2, pr

Excessive output truncated after 524337 bytes.

e: 342, t: 11, print agent 1 result: (s = 27, a = :right, r = -1, sp = 28, t = 11)
e: 342, t: 12, print agent 1 result: (s = 28, a = :right, r = -1, sp = 29, t = 12)
e: 342, t: 13, print agent 1 result: (s = 29, a = :right, r = -1, sp = 30, t = 13)
e: 342, t: 14, print agent 1 result: (s = 30, a = :right, r = -1, sp = 31, t = 14)
e: 342, t: 15, print agent 1 result: (s = 31, a = :right, r = 10, sp = 32, t = 15)
e: 344, t: 1, print agent 1 result: (s = 17, a = :right, r = -1, sp = 18, t = 1)
e: 344, t: 2, print agent 1 result: (s = 18, a = :right, r = -1, sp = 19, t = 2)
e: 344, t: 3, print agent 1 result: (s = 19, a = :right, r = -1, sp = 20, t = 3)
e: 344, t: 4, print agent 1 result: (s = 20, a = :right, r = -1, sp = 21, t = 4)
e: 344, t: 5, print agent 1 result: (s = 21, a = :right, r = -1, sp = 22, t = 5)
e: 344, t: 6, print agent 1 result: (s = 22, a = :right, r = -1, sp = 23, t = 6)
e: 344, t: 7, print agent 1 result: (s = 23, a = :right, r = -1, sp = 24, t = 7)
e: 344, t: 8, prin